In [2]:
import h5py
import numpy as np
import pandas as pd

# Open the H5 file
h5_file = h5py.File('../Data/SMAP_L3_SM_P_E_20250401_R19240_001.h5', 'r')

# List all groups and datasets in the file
def print_structure(name, obj):
    print(name)
h5_file.visititems(print_structure)

# Access a specific dataset (replace 'dataset_name' with actual path)
# dataset = h5_file['dataset_name'][:]

# Close the file when done
h5_file.close()

Metadata
Metadata/AcquisitionInformation
Metadata/AcquisitionInformation/platform
Metadata/AcquisitionInformation/platformDocument
Metadata/AcquisitionInformation/radar
Metadata/AcquisitionInformation/radarDocument
Metadata/AcquisitionInformation/radiometer
Metadata/AcquisitionInformation/radiometerDocument
Metadata/DataQuality
Metadata/DataQuality/CompletenessOmission
Metadata/DataQuality/DomainConsistency
Metadata/DatasetIdentification
Metadata/Extent
Metadata/GridSpatialRepresentation
Metadata/GridSpatialRepresentation/Column
Metadata/GridSpatialRepresentation/GridDefinition
Metadata/GridSpatialRepresentation/GridDefinitionDocument
Metadata/GridSpatialRepresentation/Row
Metadata/Lineage
Metadata/Lineage/EASEGRID_LON_M
Metadata/Lineage/InputConfiguration
Metadata/Lineage/L2_SM_P
Metadata/Lineage/L2_SM_P_E
Metadata/Lineage/MetadataConfiguration
Metadata/Lineage/OutputConfiguration
Metadata/Lineage/RunConfiguration
Metadata/OrbitMeasuredLocation
Metadata/ProcessStep
Metadata/ProductSpe